# Wavefront and WavefrontK

The `Wavefront` object represents real-space complex wavefront data of a photon field propoagating in the `z` direction with given carrier wavelength. 

The `WavefrontK` object is essentially the same fundammental data but in k-space. Some operations are more convenient in this space.

This shows some of the basic usages of these objects.

In [ ]:
from pmd_beamphysics import Wavefront

from pmd_beamphysics.wavefront.analysis import fit_m2

from scipy.constants import c
import numpy as np
import os
import matplotlib.pyplot as plt

## Basic usage

A `Wavefront` must be initialized with a 3D field array. Here for demonstration we will make 2x2 hot pixels in the array.

Note the defaults for the other parameters are set to 1 m, and the photon energy is calculated.

In [ ]:
W = Wavefront(Ex=np.zeros((11, 11, 2)))
W.Ex[6:8, 7:9, :] = 1

W

Basic fluence and power plots are included:

In [ ]:
W.plot_fluence()

In [ ]:
W.plot_power()

Mean positions in meters:

In [ ]:
W.mean_x

In [ ]:
W.mean_x

In [ ]:
W.mean_z

RMS sizes in meters:

In [ ]:
W.sigma_x

In [ ]:
W.sigma_y

In [ ]:
W.sigma_z

The time-averaged field energy in J:

In [ ]:
W.energy

# Gaussian pulse 

Gaussian pulses can be instantiated with the `from_gaussian` class method. Here we will make a Gaussian at a waist `w0 = 2 * sigma_x = 100 µm`:

In [ ]:
W = Wavefront.from_gaussian(
    shape=(101, 101, 51),
    dx=10e-6,
    dy=10e-6,
    dz=10e-6,
    wavelength=1e-9,
    sigma0=100e-6,
    sigma_z=50e-6,
)

W.plot_fluence()

In [ ]:
W.plot_power()

Check that the energy is what we requested:

In [ ]:
W.energy

Check the sums of the fluence profiles agree with this:

In [ ]:
Fx = W.fluence_profile_x
np.sum(Fx) * W.dx

In [ ]:
Fy = W.fluence_profile_y
np.sum(Fy) * W.dy

Check that the summed power agrees with the energy

In [ ]:
P = W.power
np.sum(P) * W.dz / c

Check the rms waist size is 100 µm that we originally requested:

In [ ]:
assert np.isclose(W.sigma_x, 100e-6)

## Drift propagation

The `drift_wavefront` function will propagate a `Wavefront` using Frenel propagation with FFT convolutions. Here we will propagate to 100 m. 

In [ ]:
W100 = W.drift(100)
W100.plot_fluence()

In [ ]:
%%time
Zlist = np.linspace(0, 100, 20)
Wlist = [W.drift(z) for z in Zlist]

sizes = np.array([w.sigma_x for w in Wlist])
sizes

In [ ]:
# RMS waist size
sigma_x0 = W.sigma_x

# Corresponding Rayleigh length
zR = np.pi * 4 * sigma_x0**2 / W.wavelength

expected_w = sigma_x0 * np.sqrt(1 + (Zlist / zR) ** 2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(Zlist, 1e6 * expected_w, label="expected")
ax.plot(Zlist, 1e6 * sizes, "--", label="propagated")

ax.set_xlabel(r"$z$ (m)")

ax.set_ylabel(r"$\sigma_x$ (µm)")
plt.legend()

## Drift with curvature correction

The default `drift_wavefont` does not resize the grid. Alternatively, you can specify a `curvature` in 1/m in the propagation that will resize the grid to help keep the spot within the domain. 

See the `advanced_drift` notebook for more details.


In [ ]:
W100_advancd = W.drift(100, curvature=1 / 100)
W100_advancd.plot_fluence()

# Estimate curvature

This will estimate the curvature of a wavefront by fitting the complex phase across an axis in the 3D field array.

By default, the z slice with the highest power, and then vertical slice with the highest energy density, are chosen.

The `plot` option will display a plot to examine the fit.

In [ ]:
W100.estimate_curvature(plot=True)

In [ ]:
W100.estimate_curvature(axis="y", plot=True)

# $M^2$ fit

The beam size squared equation is given by:

$$
\sigma^2(z) = \sigma_{0}^2 \left[ 1 + \left(\frac{M^2 \lambda}{4\pi \sigma_{0}^2} (z - z_0) \right)^2 \right]
$$

Where:
- $\sigma(z)$: RMS beam size at position \(z\).
- $\sigma_{0}$: Minimum RMS beam size at position (at the waist).
- $z_0$: Position of the beam waist.
- $z$: Position along the propagation axis.
- $M^2$: Beam quality factor.
- $\lambda$: Wavelength of the light.


In [ ]:
fit_m2(Zlist, sizes, wavelength=W.wavelength, plot=True)

# K-space 

We transform to k-space according to the Fourier transform convention:

Ẽ(kx,ky,kz) = 1/(2π)^(2/3) ∫∫∫ E(x,y,z) exp(-i kx x) exp(-i ky y) exp(-i kz z) dx dy dz

with units in V * m^2


In [ ]:
Wk = W.to_kspace()

Wk.plot_spectral_intensity()

The photon spectrum can be plotted:

In [ ]:
Wk.plot_photon_energy_spectrum()

This corresponds with the data:

In [ ]:
Wk.photon_energy_vec, Wk.photon_energy_spectrum

The energy should be the same as in r-space:

In [ ]:
Wk.energy

These are the rms angular sizes in radians:

In [ ]:
Wk.sigma_thetax

In [ ]:
Wk.sigma_thetay

Check that the angular sizes are preseved with the previous propagation

In [ ]:
[float(w.to_kspace().sigma_thetax) for w in Wlist]

In [ ]:
1 / W.sigma_x / W.k0 / 2

In [ ]:
Wk.sigma_kx

In [ ]:
1 / W.sigma_x / 2

# Pad, Crop, and Auto-crop

It is often needed to pad the field arrays with zeros to account for propagation or for better spectral resolution. Both `Wavefront` and `WavefrontK` have methods to do this.

Additionally the `.auto_crop(threshold)` looks at intensity profiles to automatically crop. 

In [ ]:
W.pad(100, (0, 20)).plot_fluence()

In [ ]:
W.shape

In [ ]:
W.pad(10, 20, (2, 5)).shape

In [ ]:
W.crop().shape

In [ ]:
W.pad(10, 20, (2, 5)).crop(10, 20, (2, 5)).shape

In [ ]:
W.pad(10, 20, (2, 5)).auto_crop().shape

In [ ]:
W.auto_crop(threshold=1e-3).shape

# Conversion

## Genesis4

The `.write_genesis4` method will write to a Genesis4 native HDF5 file. Such files also be read in with `.from_genesis4`. 

In [ ]:
W.write_genesis4("genesis4_field.h5")

In [ ]:
W2 = Wavefront.from_genesis4("genesis4_field.h5")

Check that the results are close:

In [ ]:
np.allclose(W.Ex, W2.Ex)

In [ ]:
# Cleanup
os.remove("genesis4_field.h5")